# PANAS: Positive and Negative Affect Schedule

This notebook implements the PANAS scale for measuring positive and negative affect in brain-mediated music research.

## Background

The **PANAS** (Watson, Clark, & Tellegen, 1988) is one of the most widely used measures of mood and emotion.

### Psychometric Properties
- **Reliability**: High internal consistency (α > 0.85 for both scales)
- **Validity**: Well-validated across cultures and contexts
- **Sensitivity**: Responsive to mood changes
- **Independence**: PA and NA are largely independent dimensions

### Use Cases in BrainJam
- Pre/post intervention mood assessment
- Correlating affect with neural markers
- Validating emotional impact of music systems
- Monitoring psychological safety

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("Libraries loaded successfully")

## 1. PANAS Items

The PANAS consists of 20 mood descriptors (10 positive, 10 negative).

In [ ]:
# PANAS items
panas_items = {
    # Positive Affect items
    1: "Interested",
    2: "Distressed",
    3: "Excited",
    4: "Upset",
    5: "Strong",
    6: "Guilty",
    7: "Scared",
    8: "Hostile",
    9: "Enthusiastic",
    10: "Proud",
    11: "Irritable",
    12: "Alert",
    13: "Ashamed",
    14: "Inspired",
    15: "Nervous",
    16: "Determined",
    17: "Attentive",
    18: "Jittery",
    19: "Active",
    20: "Afraid"
}

# Item categorization
positive_items = [1, 3, 5, 9, 10, 12, 14, 16, 17, 19]
negative_items = [2, 4, 6, 7, 8, 11, 13, 15, 18, 20]

# Instructions (can be modified for different time frames)
instructions = """
This scale consists of a number of words that describe different feelings and emotions.
Read each item and then mark the appropriate answer in the space next to that word.
Indicate to what extent you feel this way RIGHT NOW, that is, at the present moment.

Use the following scale to record your answers:
1 = Very slightly or not at all
2 = A little
3 = Moderately
4 = Quite a bit
5 = Extremely
"""

print("PANAS items loaded:")
print(f"Positive Affect items: {len(positive_items)}")
print(f"Negative Affect items: {len(negative_items)}")

## 2. Scoring Functions

In [ ]:
def score_panas(responses):
    """
    Score PANAS responses.
    
    Parameters:
    -----------
    responses : dict or pd.Series
        Item numbers (1-20) as keys, ratings (1-5) as values
    
    Returns:
    --------
    dict with 'PA' (Positive Affect) and 'NA' (Negative Affect) scores
    """
    if isinstance(responses, pd.Series):
        responses = responses.to_dict()
    
    # Calculate scores (range: 10-50 for each)
    pa_score = sum(responses[i] for i in positive_items if i in responses)
    na_score = sum(responses[i] for i in negative_items if i in responses)
    
    return {
        'PA': pa_score,
        'NA': na_score,
        'PA_mean': pa_score / len(positive_items),
        'NA_mean': na_score / len(negative_items)
    }

def interpret_panas(scores):
    """
    Provide interpretation of PANAS scores.
    """
    interpretation = []
    
    # Positive Affect interpretation (based on normative data)
    if scores['PA'] < 20:
        interpretation.append("Low positive affect")
    elif scores['PA'] < 35:
        interpretation.append("Moderate positive affect")
    else:
        interpretation.append("High positive affect")
    
    # Negative Affect interpretation
    if scores['NA'] < 15:
        interpretation.append("Low negative affect")
    elif scores['NA'] < 25:
        interpretation.append("Moderate negative affect")
    else:
        interpretation.append("High negative affect")
    
    return " | ".join(interpretation)

# Example scoring
example_responses = {i: np.random.randint(1, 6) for i in range(1, 21)}
example_scores = score_panas(example_responses)
print("Example PANAS scores:")
print(f"Positive Affect: {example_scores['PA']} (mean: {example_scores['PA_mean']:.2f})")
print(f"Negative Affect: {example_scores['NA']} (mean: {example_scores['NA_mean']:.2f})")
print(f"Interpretation: {interpret_panas(example_scores)}")

## 3. Simulated Data Example

Let's simulate PANAS data from a pre/post intervention study.

In [ ]:
# Set seed for reproducibility
np.random.seed(42)

# Simulate data for 30 participants, pre and post intervention
n_participants = 30

# Pre-intervention: moderate affect
pre_pa = np.random.normal(loc=28, scale=6, size=n_participants)
pre_na = np.random.normal(loc=18, scale=5, size=n_participants)

# Post-intervention: increased PA, decreased NA
post_pa = pre_pa + np.random.normal(loc=5, scale=3, size=n_participants)  # Increase
post_na = pre_na - np.random.normal(loc=3, scale=2, size=n_participants)  # Decrease

# Ensure valid ranges (10-50)
pre_pa = np.clip(pre_pa, 10, 50)
post_pa = np.clip(post_pa, 10, 50)
pre_na = np.clip(pre_na, 10, 50)
post_na = np.clip(post_na, 10, 50)

# Create DataFrame
data = pd.DataFrame({
    'participant_id': range(1, n_participants + 1),
    'pre_PA': pre_pa,
    'pre_NA': pre_na,
    'post_PA': post_pa,
    'post_NA': post_na
})

# Calculate change scores
data['PA_change'] = data['post_PA'] - data['pre_PA']
data['NA_change'] = data['post_NA'] - data['pre_NA']

print("Simulated PANAS data:")
print(data.describe())

## 4. Visualization

In [ ]:
# Pre/Post comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Positive Affect
axes[0].scatter(data['pre_PA'], data['post_PA'], alpha=0.6, s=80, color='green')
axes[0].plot([10, 50], [10, 50], 'k--', lw=2, alpha=0.3, label='No change')
axes[0].set_xlabel('Pre-intervention PA', fontsize=12)
axes[0].set_ylabel('Post-intervention PA', fontsize=12)
axes[0].set_title('Positive Affect: Pre vs Post', fontsize=14, fontweight='bold')
axes[0].set_xlim(10, 50)
axes[0].set_ylim(10, 50)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Negative Affect
axes[1].scatter(data['pre_NA'], data['post_NA'], alpha=0.6, s=80, color='red')
axes[1].plot([10, 50], [10, 50], 'k--', lw=2, alpha=0.3, label='No change')
axes[1].set_xlabel('Pre-intervention NA', fontsize=12)
axes[1].set_ylabel('Post-intervention NA', fontsize=12)
axes[1].set_title('Negative Affect: Pre vs Post', fontsize=14, fontweight='bold')
axes[1].set_xlim(10, 50)
axes[1].set_ylim(10, 50)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Points above the diagonal line indicate increased affect.")
print("Points below the diagonal line indicate decreased affect.")

In [ ]:
# Distribution of change scores
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PA change
axes[0].hist(data['PA_change'], bins=15, color='green', alpha=0.7, edgecolor='black')
axes[0].axvline(0, color='red', linestyle='--', linewidth=2, label='No change')
axes[0].axvline(data['PA_change'].mean(), color='darkgreen', linestyle='-', linewidth=2, 
                label=f'Mean: {data["PA_change"].mean():.2f}')
axes[0].set_xlabel('PA Change (Post - Pre)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Change in Positive Affect', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# NA change
axes[1].hist(data['NA_change'], bins=15, color='red', alpha=0.7, edgecolor='black')
axes[1].axvline(0, color='green', linestyle='--', linewidth=2, label='No change')
axes[1].axvline(data['NA_change'].mean(), color='darkred', linestyle='-', linewidth=2,
                label=f'Mean: {data["NA_change"].mean():.2f}')
axes[1].set_xlabel('NA Change (Post - Pre)', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Change in Negative Affect', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 5. Statistical Analysis

In [ ]:
# Paired t-tests
pa_ttest = stats.ttest_rel(data['post_PA'], data['pre_PA'])
na_ttest = stats.ttest_rel(data['post_NA'], data['pre_NA'])

# Effect sizes (Cohen's d for paired samples)
def cohens_d_paired(pre, post):
    diff = post - pre
    return diff.mean() / diff.std()

pa_d = cohens_d_paired(data['pre_PA'], data['post_PA'])
na_d = cohens_d_paired(data['pre_NA'], data['post_NA'])

print("=" * 60)
print("STATISTICAL ANALYSIS: Pre-Post Intervention Comparison")
print("=" * 60)

print("\n📊 POSITIVE AFFECT")
print(f"Pre-intervention:  M = {data['pre_PA'].mean():.2f}, SD = {data['pre_PA'].std():.2f}")
print(f"Post-intervention: M = {data['post_PA'].mean():.2f}, SD = {data['post_PA'].std():.2f}")
print(f"Change:            M = {data['PA_change'].mean():.2f}, SD = {data['PA_change'].std():.2f}")
print(f"\nPaired t-test: t({len(data)-1}) = {pa_ttest.statistic:.3f}, p = {pa_ttest.pvalue:.4f}")
print(f"Cohen's d: {pa_d:.3f}")
if pa_ttest.pvalue < 0.05:
    print("✓ Significant increase in positive affect")

print("\n📊 NEGATIVE AFFECT")
print(f"Pre-intervention:  M = {data['pre_NA'].mean():.2f}, SD = {data['pre_NA'].std():.2f}")
print(f"Post-intervention: M = {data['post_NA'].mean():.2f}, SD = {data['post_NA'].std():.2f}")
print(f"Change:            M = {data['NA_change'].mean():.2f}, SD = {data['NA_change'].std():.2f}")
print(f"\nPaired t-test: t({len(data)-1}) = {na_ttest.statistic:.3f}, p = {na_ttest.pvalue:.4f}")
print(f"Cohen's d: {na_d:.3f}")
if na_ttest.pvalue < 0.05:
    print("✓ Significant decrease in negative affect")

print("\n" + "=" * 60)

## 6. Reliability Analysis

In [ ]:
# Simulate item-level data for reliability analysis
def simulate_item_responses(n_participants, scale_mean, scale_std, n_items=10):
    """Simulate correlated item responses for reliability analysis"""
    # Generate latent trait scores
    latent_scores = np.random.normal(scale_mean/n_items, scale_std/n_items, n_participants)
    
    # Generate item responses with some item-specific variation
    items = []
    for i in range(n_items):
        item_responses = latent_scores + np.random.normal(0, 0.3, n_participants)
        item_responses = np.clip(item_responses, 1, 5)  # Ensure valid range
        items.append(item_responses)
    
    return np.array(items).T

# Simulate pre-intervention item-level data
pa_items_data = simulate_item_responses(n_participants, data['pre_PA'].mean(), 
                                        data['pre_PA'].std(), n_items=10)
na_items_data = simulate_item_responses(n_participants, data['pre_NA'].mean(), 
                                        data['pre_NA'].std(), n_items=10)

# Calculate Cronbach's alpha manually
def cronbach_alpha(item_data):
    """Calculate Cronbach's alpha for internal consistency"""
    n_items = item_data.shape[1]
    item_variances = item_data.var(axis=0, ddof=1)
    total_variance = item_data.sum(axis=1).var(ddof=1)
    
    alpha = (n_items / (n_items - 1)) * (1 - item_variances.sum() / total_variance)
    return alpha

pa_alpha = cronbach_alpha(pa_items_data)
na_alpha = cronbach_alpha(na_items_data)

print("RELIABILITY ANALYSIS")
print("=" * 60)
print(f"\nPositive Affect Scale: α = {pa_alpha:.3f}")
if pa_alpha >= 0.9:
    print("  ✓ Excellent internal consistency")
elif pa_alpha >= 0.8:
    print("  ✓ Good internal consistency")
elif pa_alpha >= 0.7:
    print("  ✓ Acceptable internal consistency")
    
print(f"\nNegative Affect Scale: α = {na_alpha:.3f}")
if na_alpha >= 0.9:
    print("  ✓ Excellent internal consistency")
elif na_alpha >= 0.8:
    print("  ✓ Good internal consistency")
elif na_alpha >= 0.7:
    print("  ✓ Acceptable internal consistency")

print("\n" + "=" * 60)
print("Note: Typical PANAS reliability (α) is > 0.85 for both scales")

## 7. Integration with Neural Data

Example: Correlating PANAS scores with EEG frontal alpha asymmetry.

In [ ]:
# Simulate EEG frontal alpha asymmetry (left - right)
# Positive values = relatively more left activity (associated with positive affect)
# Negative values = relatively more right activity (associated with negative affect)

# Create asymmetry scores correlated with affect
alpha_asymmetry = (data['post_PA'] - data['post_NA']) / 40 + np.random.normal(0, 0.3, n_participants)

# Correlations
pa_corr = stats.pearsonr(data['post_PA'], alpha_asymmetry)
na_corr = stats.pearsonr(data['post_NA'], alpha_asymmetry)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PA vs asymmetry
axes[0].scatter(data['post_PA'], alpha_asymmetry, alpha=0.6, s=80, color='green')
z = np.polyfit(data['post_PA'], alpha_asymmetry, 1)
p = np.poly1d(z)
axes[0].plot(data['post_PA'], p(data['post_PA']), "r--", linewidth=2)
axes[0].set_xlabel('Positive Affect Score', fontsize=12)
axes[0].set_ylabel('Frontal Alpha Asymmetry', fontsize=12)
axes[0].set_title(f'PA vs EEG Asymmetry\nr = {pa_corr[0]:.3f}, p = {pa_corr[1]:.4f}', 
                  fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# NA vs asymmetry
axes[1].scatter(data['post_NA'], alpha_asymmetry, alpha=0.6, s=80, color='red')
z = np.polyfit(data['post_NA'], alpha_asymmetry, 1)
p = np.poly1d(z)
axes[1].plot(data['post_NA'], p(data['post_NA']), "b--", linewidth=2)
axes[1].set_xlabel('Negative Affect Score', fontsize=12)
axes[1].set_ylabel('Frontal Alpha Asymmetry', fontsize=12)
axes[1].set_title(f'NA vs EEG Asymmetry\nr = {na_corr[0]:.3f}, p = {na_corr[1]:.4f}',
                  fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nBrain-Behavior Correlations:")
print(f"PA - Asymmetry: r = {pa_corr[0]:.3f}, p = {pa_corr[1]:.4f}")
print(f"NA - Asymmetry: r = {na_corr[0]:.3f}, p = {na_corr[1]:.4f}")

## 8. Practical Recommendations

### When to Use PANAS

✅ **Good for:**
- Pre/post intervention comparisons
- General mood assessment
- Correlating with neural markers
- Repeated measures (can be given multiple times)
- Quick assessment (2-3 minutes)

❌ **Not ideal for:**
- Specific emotions (use AESTHEMOS instead)
- Continuous monitoring (use SAM instead)
- Children (language may be too complex)
- Very frequent assessment (may cause response fatigue)

### Administration Tips

1. **Time frame**: Adjust instructions for "right now", "during the past week", or "in general"
2. **Order**: Counterbalance with other measures to avoid order effects
3. **Context**: Consider environmental factors that might affect mood
4. **Baseline**: Always collect baseline data for comparison
5. **Follow-up**: Consider longer-term follow-up to assess lasting effects

### Interpretation Guidelines

- **Clinical significance**: Changes of ≥5 points often considered meaningful
- **Effect size**: Cohen's d of 0.2 (small), 0.5 (medium), 0.8 (large)
- **Individual differences**: Wide variation is normal; focus on within-person change
- **Independence**: PA and NA are separate dimensions, not opposite ends of one scale

## References

Watson, D., Clark, L. A., & Tellegen, A. (1988). Development and validation of brief measures of positive and negative affect: The PANAS scales. *Journal of Personality and Social Psychology*, 54(6), 1063-1070.

Crawford, J. R., & Henry, J. D. (2004). The Positive and Negative Affect Schedule (PANAS): Construct validity, measurement properties and normative data in a large non‐clinical sample. *British Journal of Clinical Psychology*, 43(3), 245-265.